In [1]:
import numpy as np
import pandas as pd

from PIL import Image 
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import os
import re
import cv2

import skimage
from sklearn.base import BaseEstimator, TransformerMixin
from skimage.feature import hog

from scipy.sparse import csr_matrix
import h5py


In [3]:
# chemin des images
folder_path = 'C:/Users/Nans/OneDrive/Documents/Rakuten Project/images/images/image_train'


In [33]:
def img_cropping(img):

    largeur_fixe = 224
    hauteur_fixe = 224
    
    # Convertir l'image en niveaux de gris
    image_gris = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Inverser les niveaux de gris
    image_inversee = cv2.bitwise_not(image_gris)

    # Trouver les contours dans l'image
    contours, hierarchy = cv2.findContours(image_inversee, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        
        # Trouver le contour le plus externe
        contour_externe = max(contours, key=cv2.contourArea)

        # Trouver les coordonnées du rectangle englobant le contour externe
        x, y, w, h = cv2.boundingRect(contour_externe)

        # Rogner l'image en utilisant les coordonnées du rectangle englobant
        cropped_image = img[y:y+h, x:x+w]

        cropped_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB)

        # Redimensionner l'image pour qu'elle ait la même taille que les autres
        cropped_image_resized = cv2.resize(cropped_image, (largeur_fixe, hauteur_fixe))

        return np.array(cropped_image_resized)

    else:
        # Si aucun contour n'est trouvé, retourner None
        return None



In [36]:

# Obtenez la liste des fichiers dans le dossier
img_in_folder = os.listdir(folder_path)

# Parcourez chaque fichier dans le dossier
batch_size = 10000
num_batches = len(img_in_folder) // batch_size + 1
#num_batches = 2

for batch_idx in range(num_batches):
    start_idx = batch_idx * batch_size
    end_idx = min((batch_idx + 1) * batch_size, len(img_in_folder))
    
    batch_files = img_in_folder[start_idx:end_idx]

    # Initialisez des listes pour stocker les identifiants des images et des produits
    image_ids = []
    product_ids = []
    image_vectors = []  
    
    # Traitement du lot d'images
    for img in batch_files:

        # Utilisez une expression régulière pour extraire les identifiants d'image et de produit
        match = re.match(r'image_(\d+)_product_(\d+)\.jpg', img)
        if match:
            image_id = match.group(1)
            product_id = match.group(2)
        
            # Lire l'image avec OpenCV
            img_path = os.path.join(folder_path, img)
            image = cv2.imread(img_path)

            # Fonction d'image cropping
            cropped_img = img_cropping(image)
        
            # Assurez-vous que l'image existe
            if cropped_img is not None:           
                # Ajoutez les identifiants et le vecteur d'image à leurs listes respectives
                image_ids.append(int(image_id))
                product_ids.append(int(product_id))
                image_vectors.append(cropped_img)
            
    # Convertir les listes en tableaux NumPy
    image_vectors = np.array(image_vectors)
    image_ids = np.array(image_ids)
    product_ids = np.array(product_ids)


    # Créer un fichier HDF5 et y enregistrer un tableau NumPy
    with h5py.File('data_' + str(batch_idx) + 'comp.h5', 'w') as hf:
        hf.create_dataset('X_train', data=image_vectors, compression='gzip')
        hf.create_dataset('image_ids', data=image_ids, compression='gzip')
        hf.create_dataset('product_ids', data=product_ids, compression='gzip')


In [29]:
batch_idx

1